In [ ]:
import tifffile
import imageio
from cellpose import models
import numpy as np
from cellstitch.pipeline import *
from cellstitch.evaluation import *
import pandas as pd
import h5py

In [ ]:
dataset = "Valve"

# Train cellpose model from scratch

In [ ]:
for i in range(70): 
    filename = "%s_%02d" % (dataset, i)
    img = np.load("../DATA/%s/%s.npy" % (dataset, filename))
    labels = np.load("../DATA/%s/%s_masks.npy" % (dataset, filename))
    depth = img.shape[0] 
    
    for i in range(depth): 
        imageio.imwrite("../DATA/%s/cellpose_train/%s_%s.tif" % (dataset, filename, i), img[i])
        imageio.imwrite("../DATA/%s/cellpose_train/%s_%s_masks.tif" % (dataset, filename, i), labels[i])

`python -m cellpose --train --use_gpu --dir ../DATA/<dataset>/cellpose_train --pretrained_model None --n_epochs 100  --verbose`

###  Generate cellpose3d results

In [ ]:
model_dir = '../DATA/Valve/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2023_05_30_20_25_23.614818'

In [ ]:
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [ ]:
for i in range(70, 100): 
    test_filename = "%s_%02d" % (dataset, i)
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/%s.npy" % (dataset, test_filename))
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0]) 
    np.save("./results/%s/cellpose3d/%s.npy" % (dataset, test_filename), masks) 

### Generate cellpose2d results

In [ ]:
for i in range(70, 100): 
    test_filename = "%s_%02d" % (dataset, i)
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/%s.npy" % (dataset, test_filename))
    masks, _, _ = model.eval(list(img), do_3D=False, flow_threshold=flow_threshold, channels = [0,0])
    masks = cp_utils.stitch3D(np.array(masks))

    np.save("./results/%s/cellpose2d/%s.npy" % (dataset, test_filename), masks) 

### Generate cellstitch results

In [ ]:
for i in range(70, 100): 
    test_filename = "%s_%02d" % (dataset, i)
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/%s.npy" % (dataset, test_filename))
    
    cellstitch, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0])
    cellstitch = np.array(cellstitch)

    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)

    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/%s/cellstitch/%s.npy" % (dataset, test_filename), cellstitch)  

# Benchmark results

In [ ]:
ap_threshold = 0.5
method = "cellpose2d"

In [ ]:
data = [] 
for i in range(70, 100): 
    filename = "%s_%02d" % (dataset, i)
    labels = np.load('../DATA/%s/%s_masks.npy' % (dataset, filename))

    masks = np.load("./results/%s/%s/%s.npy" % (dataset, method, filename))


    ap, tp, fp, fn = average_precision(labels, masks, ap_threshold)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    row = [
        filename, 
        ap, 
        tp, 
        fp, 
        fn, 
        precision,
        recall
    ]

    data.append(row)

df = pd.DataFrame(data, columns=[
    "filename", 
    "ap", 
    "tp", 
    "fp", 
    "fn",
    "precision",
    "recall"
])

df.to_csv("./results/%s/%s.csv" % (dataset, method), index=False)